# 🏥 Fall Risk Detection in Nursing Notes Using ClinicalBERT

This project demonstrates the use of [ClinicalBERT](https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT) to classify nursing notes for fall risk. The dataset used is a simulated collection of EHR-style notes labeled for fall risk presence or absence.

This project simulates how natural language processing (NLP) can assist in patient safety surveillance — particularly in support of CMS-reportable fall-risk metrics.

**Key Goals:**
- Use a pretrained ClinicalBERT model for classification
- Tokenize and format unstructured clinical text
- Fine-tune on a simulated dataset of nursing documentation
- Evaluate classification performance


# Step 1: Load and Preview the Dataset

In [ ]:
import pandas as pd

# Replace 'dataset.csv' with your actual filename
df = pd.read_csv("dataset_expanded.csv")

from datasets import Dataset

dataset = Dataset.from_pandas(df)

# Step 2: Load ClinicalBERT and Tokenize the Notes

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Load ClinicalBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Tokenize the dataset
def tokenize(example):
    return tokenizer(
        example['text'],
        padding='max_length',
        truncation=True,
        max_length=512  # Explicitly define max_length
    )

dataset = dataset.map(tokenize)

# Step 3: Load Model and Train

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "emilyalsentzer/Bio_ClinicalBERT",
    num_labels=2
)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

trainer.train()

# Step 4: Evaluate the Model

In [ ]:
from transformers import AutoTokenizer

# Load the ClinicalBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# Define a function to tokenize your dataset text column (replace 'text' if your column is named differently)
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True, max_length=128)

# Apply the tokenizer to the entire dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Remove columns you don't need (like the original text) for training
tokenized_dataset = tokenized_dataset.remove_columns(['text'])

# Set format for TensorFlow 
tokenized_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])


# Step 5: Prepare Training and Evaluation

In [ ]:
from datasets import ClassLabel
# Split the dataset into training and test sets (80/20)
train_test = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test['train']
eval_dataset = train_test['test']

# Step 6: Load ClinicalBERT and Train the Model

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load the ClinicalBERT model for binary classification
model = AutoModelForSequenceClassification.from_pretrained(
    "emilyalsentzer/Bio_ClinicalBERT",
    num_labels=2
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Define a simple compute_metrics function (optional but useful)
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Initialize the Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

# Step 7: Save and Evaluate the Model

In [ ]:
# Save the Model
trainer.save_model("./clinicalbert_fall_risk_model")

# Evaluate the Model
trainer.evaluate()

# Step 8: Tokenize and Train

# Step 8.1: Load and Convert to Dataset

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("dataset_expanded.csv")  # adjust path as needed
dataset = Dataset.from_pandas(df)

# Step 8.2: Tokenize

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Step 8.3: Train/Test Split

In [ ]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# Step 8.4: Model and Trainer Setup

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model = AutoModelForSequenceClassification.from_pretrained(
    "emilyalsentzer/Bio_ClinicalBERT", num_labels=2
)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Step 8.5: Train

In [ ]:
trainer.train()

# Step 8.6: Save Trained Model

In [ ]:
trainer.save_model("./clinicalbert_fallrisk_model")

## ✅ Conclusion

This notebook demonstrates how ClinicalBERT — a transformer model pretrained on real clinical notes — can be fine-tuned on simulated nursing documentation to classify fall risk. This method is promising for tasks such as:

- Identifying documentation patterns tied to fall risks
- Supporting CMS quality metric reporting (e.g., inpatient falls)
- Assisting healthcare administrators in early warning or dashboard systems

Next steps may include:
- Expanding the dataset for better generalizability
- Exploring multi-label classification (e.g., fall + pressure injury risk)
- Deploying the model in a lightweight dashboard using Streamlit

*Authored by Larry L. Splane, RN, MBA, VA-BC*
